In [2]:
import pandas as pd
import numpy as np

import statistics

import re

import nltk

import kiwipiepy
from kiwipiepy import Kiwi # 형태소 분석
from kiwipiepy.utils import Stopwords # kiwi 불용어

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm
import time

In [3]:
def Rep_text(text): # 단어 통일 시키기
    text = str(text)
    pattern = '(\n){1,}' 
    text = re.sub(pattern=pattern, repl='\n', string=text) # \n가 1번이상 사용된 경우 \n로 바꾸기
    return text

# data load

In [4]:
# 파일 열기
df = pd.read_csv('../data/pre_data.csv')

# stopword

In [5]:
# stopword
file_path = 'stopwords.txt'  # 경로 지정
with open(file_path, 'r', encoding='utf-8') as file:
    word = file.read()

word = word.split('\n') # \n 구분
# stopword 추가하기
word_lst = ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', 
            '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', 
            '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', 
            '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', 
            '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', 
            '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', 
            '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', 
            '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', 
            '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', 
            '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하다', '하면', '하지', '한하다', 
            '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] + ['클린','표현','감지','댓글','작가',
             '웹툰','네이버'] + ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

for w in word_lst:
    word.append(w)
stopwords = set(word)

# 웹툰의 장르와 웹툰 설명으로 유사한 웹툰 찾기

In [7]:
df = df[['title', 'genre', 'descript']]
df.drop_duplicates(subset=['title','descript'],inplace=True) # 중복 row 제거
df = df.reset_index().iloc[:,1:]

In [9]:
# kiwi활용 토큰화
kiwi_stop = Stopwords() # kiwi 불용어
kiwi = kiwipiepy.Kiwi()

def extract_nouns(text): # kiwi사용하여 토큰화
    for token in kiwi.tokenize(text, normalize_coda=True, stopwords=kiwi_stop):
        yield token.form
        
def desc(text): # 토큰화 진행 후, 불용어 처리
    text = str(text)
    words = list(extract_nouns(text))
    all_words = ""
    for word in words:
        if word not in stopwords:  # stopwords에 포함되는 단어 제거
            all_words+=f'{word} ' # 하나로 합치기
    return all_words[:-1]

def genreee(text): # #을 기준으로 장르 컬럼에 들어간 장르들 나누기
    text = str(text)
    all_genre = ""
    g = text.split('#')[1:] # 0번째는 필요없으므로 제외
    for i in g: 
        all_genre+=f'{i} ' # 하나로 합치기
    return all_genre[:-1]

df['pre_descript'] = df['descript'].apply(lambda x: desc(x))
df['pre_genre'] = df['genre'].apply(lambda x: genreee(x))

In [32]:
tfidf = TfidfVectorizer() #TF-IDF로 만들기
tfidf_des = tfidf.fit_transform(df['pre_descript']) # 설명부분
tfidf_gen = tfidf.fit_transform(df['pre_genre']) # 장르부분

tfidf_des # 138개의 웹툰, 각 웹툰은 1797개의 유니크한 단어
tfidf_gen # 138개의 웹툰, 각 웹툰은 122개의 유니크한 장르

# 두 TF-IDF 합치기
tfidf_mat = np.concatenate((tfidf_des.toarray(), tfidf_gen.toarray()), axis=1)

# cosine_similarity 구하기
cosine_sim = cosine_similarity(tfidf_mat, tfidf_mat)

웹툰 설명 부분만 사용해서 했을 경우, 장르의 유사성을 반영하지 못하는 것 같아서  
설명과 장르의 TF-IDF를 뽑은 후  
둘을 concat하여 코사인 유사도를 구해 진행하였습니다

In [33]:
# 교수님의 강의 코드를 활용하였습니다.
df['id'] = list(range(0,len(df)))

id_to_index = {k:v for k,v in zip(df['id'], df.index)}
index_to_id = {v:k for k,v in id_to_index.items()}
id_to_title = {k:v for k,v in zip(df['id'], df['title'])}
title_to_id = {v:k for k,v in id_to_title.items()}

def get_top10_similarity(web_title, cosine_sim):
    # title --> id --> index
    web_id = title_to_id[web_title]
    idx = id_to_index[web_id]
    
    sim_score = list(enumerate(cosine_sim[idx])) # sim_score = [(0, 유사도(기준영화, 영화0)), (1, 유사도(기준영화, 영화1)), ...]
    
    # 유사도에 따라 sim_score를 내림차순 정렬. sim_score = [(10, 유사도(기준영화, 영화10)), (131, 유사도(기준영화, 영화131)), ...]
    sim_score = sorted(sim_score, key = lambda x: x[1], reverse=True)
    top10_webs = sim_score[:11] # 유사도 상위 10개를 선택 (자기 자신 포함)
    
    # 유사도 상위 10개를 선택 top10_webs = [10, 131, 52, ...]
    top10_webs = [x[0] for x in top10_webs]
    print(top10_webs[1:11]) # 유사한 웹툰 id (자기자신은 제외)
    
    top10_webs = [index_to_id[x] for x in top10_webs] # 웹툰 id 산출
    top10_webs = [id_to_title[_id] for _id in top10_webs] # 웹툰 title 산출

    return top10_webs

In [21]:
# df['title'].unique()

array(['연애혁명', '여신강림', '2021 최애캐 안녕, 잘 지내니?', '열녀박씨 계약결혼뎐', '키스의 여왕',
       '그 남주와 이별하는 방법', '널 사랑하는 죽은 형', '키스 식스 센스', '놓지마 정신줄', '키미앤조이',
       '이별학', '청춘 블라썸', '연놈', '이말년씨리즈', '2020 호랑 공포 단편선', '불청객',
       '2022 서브병에 빠지다!', '별이삼샵', '마침내 사랑이에요 마왕님!', '독립일기', '내 남편과 결혼해줘',
       '풋내기들', '오직, 밝은 미래', '모노마니아', '참교육', '신이 담긴 아이', '2022 몰래보는 로맨스',
       '늑대처럼 홀로', '초인의 시대', '돌아온 여기사', 'MLB카툰', '트럼프', '여고생 드래곤',
       '그 황제가 시곗바늘을 되돌린 사연', '팔이피플', '이제야 연애', '위닝샷!', '짝사랑 마들렌', '노답소녀',
       '캐슬', '화산귀환', '2011 미스테리 단편', '마녀와 용의 신혼일기', '모두에게 완자가',
       '신화급 귀속 아이템을 손에 넣었다', '앵무살수', '별난식당', '퍼니게임', '물어보는 사이', '뷰티풀 군바리',
       '신의 탑', '퀘스트지상주의', '똑 닮은 딸', '백수세끼', '장씨세가 호위무사', '소녀의 세계',
       '윈드브레이커', '김부장', '사변괴담', '하북팽가 막내아들', '올가미', '궤짝', '내가 키운 S급들',
       '마루는 강쥐', '대학원 탈출일지', '사신소년', '한림체육관', '멸망 이후의 세계', '하루만 네가 되고 싶어',
       '삼국지톡', '전지적 독자 시점', '닥터앤닥터 병원일기', '격기3반', '튜토리얼 탑의 고인물',
       '내가 죽기로 결심한 것은', '나쁜사람', '운명을 보는 회사원', '캐슬2:만인지상', '백XX',
       '66666년 만에 환생한 흑마법사

In [34]:
choice_web = '연애혁명' # 웹툰 선택
web_lst = get_top10_similarity(choice_web, cosine_sim) # 선택한 웹툰과 유사한 top10 웹툰

pd.merge(pd.DataFrame({'title':web_lst}),df, on='title', how='left')[['title','genre','descript']]

[12, 10, 23, 2, 11, 111, 22, 55, 104, 17]


,title,genre,descript
0,연애혁명,#드라마#다정남#혐관로맨스#하이틴#학원로맨스#완결무료#완결드라마,평범하면서 금사빠인 고등학생 순정남 공주영은 까칠하고 차가운 여학생 왕자림을 보고 ...
1,연놈,#드라마#학원로맨스#완결무료#완결드라마,"중2 여름, 가지 말라는 말도 좋아했다는 말도 하지 못한 채 보낸 그녀가 3년 만에..."
2,이별학,#드라마#완결무료#완결드라마,서른 살까지 모솔로 살다 겨우 첫 연애를 시작한 '고장가'는3년을 만나도 여자친구 ...
3,모노마니아,#드라마#완결무료#완결드라마,사랑스러운 외모와 밝은 성격으로 인기가 많은 '지우'. 하루가 멀다 하고 고백을 받...
4,"2021 최애캐 안녕, 잘 지내니?",#드라마#완결무료#완결드라마,"""잘 지내..? 오늘따라 네가 더 생각난다.""함께 웃고 울었던 웹툰 속 나의 최애캐..."
5,청춘 블라썸,#로맨스#드라마#학원로맨스#하이틴#캠퍼스로맨스#성장드라마,“푸르지 않아도 괜찮아!”이제 막 청춘을 시작하고 있는 아이들.그리고 마냥 아름답지...
6,작전명 순정,#로맨스#하이틴,내 남자친구와 절친의 키스를 목격했다. 그것도 하필이면 재수 없는 같은 반 남자애 ...
7,"오직, 밝은 미래",#드라마#타임슬립#성장드라마#완결무료#완결드라마,"'어제'로 돌아갈 수 있다면, 당신은 어떤 선택을 하시겠습니까?취업도, 연애도, 전..."
8,소녀의 세계,#드라마#학원로맨스#하이틴#직진남#삼각관계#까칠남#감성드라마,완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나여러 갈등을 함께 겪으...
9,재혼 황후,#로맨스#다정남#후회물#궁중로맨스#혐관로맨스#직진남#이세계#역하렘,동대제국의 완벽한 황후였던 나비에.황제인 남편이 정부를 황후로 만들려는 것을 알고 ...


In [27]:
choice_web = '재혼 황후' # 웹툰 선택
web_lst = get_top10_similarity(choice_web, cosine_sim) # 선택한 웹툰과 유사한 top10 웹툰

pd.merge(pd.DataFrame({'title':web_lst}),df, on='title', how='left')[['title','genre','descript']]

[68, 33, 20, 0, 87, 55, 48, 40, 111, 60]


,title,genre,descript
0,재혼 황후,#로맨스#다정남#후회물#궁중로맨스#혐관로맨스#직진남#이세계#역하렘,동대제국의 완벽한 황후였던 나비에.황제인 남편이 정부를 황후로 만들려는 것을 알고 ...
1,하루만 네가 되고 싶어,#로맨스#최강자전#직진남#이세계#역하렘#삼각관계,"'완벽한 인생이었다, 그 애가 나타나기 전까지는'반전에 반전을 거듭하는 치밀한 궁중..."
2,그 황제가 시곗바늘을 되돌린 사연,#로맨스#후회물#혐관로맨스#회귀#구원서사#소설원작#완결로맨스,황후 리지가 숨을 거둔 뒤에야 황제 레온은 자신의 어리석음을 깨닫고 후회했지만 이미...
3,내 남편과 결혼해줘,#로맨스#재벌#다정남#후회물#고자극로맨스#회귀#선결혼후연애#사내연애,하나뿐인 절친과 바람이 난 남편. 그것도 모자라 시한부인 나를 죽여버렸다. 그렇게 ...
4,연애혁명,#드라마#다정남#혐관로맨스#하이틴#학원로맨스#완결무료#완결드라마,평범하면서 금사빠인 고등학생 순정남 공주영은 까칠하고 차가운 여학생 왕자림을 보고 ...
5,버려진 나의 최애를 위하여,#로맨스#소설원작,"""내 최애를 살려내!""N회차 정주행할 만큼 애정하던 로판 소설에 빙의한 헤스티아,엑..."
6,소녀의 세계,#드라마#학원로맨스#하이틴#직진남#삼각관계#까칠남#감성드라마,완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나여러 갈등을 함께 겪으...
7,물어보는 사이,#로맨스#아이돌#계약연애#직진남#인외존재#연예계#삼각관계#뱀파이어,뱀파이어에게 물리면 건강해진다?!두통으로 의도치 않게 비호감 연예인이 된 이채이.우...
8,화산귀환,#무협#세계관#동양풍판타지#힘숨찐#회귀#이세계#먼치킨#소설원작,대 화산파 13대 제자.천하삼대검수 매화검존 청명. 천하를 혼란에 빠뜨린 고금제일마...
9,작전명 순정,#로맨스#하이틴,내 남자친구와 절친의 키스를 목격했다. 그것도 하필이면 재수 없는 같은 반 남자애 ...


In [28]:
choice_web = '시월드가 내게 집착한다' # 웹툰 선택
web_lst = get_top10_similarity(choice_web, cosine_sim) # 선택한 웹툰과 유사한 top10 웹툰

pd.merge(pd.DataFrame({'title':web_lst}),df, on='title', how='left')[['title','genre','descript']]

[33, 3, 106, 87, 29, 20, 74, 83, 82, 79]


,title,genre,descript
0,시월드가 내게 집착한다,#로맨스#햇살캐#집착물#계약연애#회귀#선결혼후연애#구원서사#소설원작,사랑했던 가족들이 나를 죽였다.과거로 돌아온 나는 내 목숨과 유산을 지키기로 결심하...
1,그 황제가 시곗바늘을 되돌린 사연,#로맨스#후회물#혐관로맨스#회귀#구원서사#소설원작#완결로맨스,황후 리지가 숨을 거둔 뒤에야 황제 레온은 자신의 어리석음을 깨닫고 후회했지만 이미...
2,열녀박씨 계약결혼뎐,#로맨스#계약연애#퓨전사극#선결혼후연애#소설원작#완결무료#완결로맨스,"혼례 첫날밤, 갑작스럽게 서방을 잃은 열녀 박연우. 엎친 데 덮친 격으로 괴한에게 ..."
3,어쩌다보니 천생연분,#로맨스#로맨스코미디#선결혼후연애,집에 얹혀산 지 몇 년 차더라..? 올해 32살을 맞이한 한지아(일하지 않는 프리랜...
4,버려진 나의 최애를 위하여,#로맨스#소설원작,"""내 최애를 살려내!""N회차 정주행할 만큼 애정하던 로판 소설에 빙의한 헤스티아,엑..."
5,돌아온 여기사,#로맨스#소설원작#완결무료#완결로맨스,"평범한 백작가 영애로 살아가던 ""이레나"". 그녀의 가문은 반왕 ""파벨루크""로 인해 ..."
6,내 남편과 결혼해줘,#로맨스#재벌#다정남#후회물#고자극로맨스#회귀#선결혼후연애#사내연애,하나뿐인 절친과 바람이 난 남편. 그것도 모자라 시한부인 나를 죽여버렸다. 그렇게 ...
7,내가 죽기로 결심한 것은,#로맨스#폭스남#집착물#청춘로맨스#재회,태권도 국가대표 선발전을 앞두고 부상을 입은 고3 지오는 뒤늦은 사춘기를 맛보는 중...
8,천마육성,#무협#게임판타지#회귀#먼치킨#소설원작,마교의 태왕각주 ‘사마귀’의 명령으로 화산파 출목지역으로 파견나간 비객조.‘설휘’는...
9,가짜 동맹,#로맨스#소꿉친구#계약연애#학원로맨스#친구,소꿉친구와 하루아침에 커플이 되었다?!통제가 강한 부모님들을 안심 시키고 자유롭게 ...


- 기존 설명만 활용했을 때보다 좀 더 유사한 웹툰의 목록들이 뽑힌 것 같습니다.
- 총 웹툰의 종류는 138개여서 더 많은 웹툰을 사용하여 한다면 더 다양하고, 유사한 웹툰을 추천할 수 있을 것 같습니다.